In [1]:
import os
import openai
import time
import traceback
import json
from tqdm.auto import tqdm
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [4]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_base = os.environ.get("OPENAI_API_BASE")

In [ ]:
def prompt_api(model, eval_prompt1, eval_prompt2):
    for i in range(10):
        try:
            eval_step1 = openai.ChatCompletion.create(
                model=model,
                stream=False,
                messages=[{"role": "user", "content": eval_prompt1}],
            )
            break
        except Exception as exc:
            print(traceback.format_exc())
            print(exc)
            if i != 9:
                print(f"Retrying... (i = {i})")
            eval_step1 = {"choices": [{"message": {"content": "ERROR"}}]}
            time.sleep(3)
    if eval_step1["choices"][0]["message"]["content"] == "ERROR":
        eval_step2 = {"choices": [{"message": {"content": "ERROR"}}]}
        return eval_step1, eval_step2
    for i in range(10):
        try:
            eval_step2 = openai.ChatCompletion.create(
                model=model,
                stream=False,
                messages=[
                    {"role": "user", "content": eval_prompt1},
                    {"role": "assistant", "content": eval_step1.choices[0].message.content},
                    {"role": "user", "content": eval_prompt2},
                ]
            )
            break
        except Exception as exc:
            print(traceback.format_exc())
            print(exc)
            if i != 9:
                print(f"Retrying... (i = {i})")
            eval_step2 = {"choices": [{"message": {"content": "ERROR"}}]}
            time.sleep(3)
    return eval_step1, eval_step2

In [ ]:
def get_eval_prompts(answer):
    eval_prompt1 = f"""In the following exercise, the student is given a beginning of a story. The student needs to complete it into a full story.
The exercise tests the student´s language abilities and creativity. The symbol *** marks the separator between the
prescribed beginning and the student’s completion:
{answer["prompt"]}***{answer["completion"]}
Please provide your general assessment about the part written by the student (the one after the *** symbol).
Is it gramatically correct? Is it consistent with the beginning of the story? Pay special attention to whether the
student manages to complete the sentence which is split in the middle by the separator ***.
"""
    eval_prompt2 = f"""Now, grade the student’s completion in terms of grammar, creativity, consistency with the story’s beginning and
whether the plot makes sense. Moreover, please provide your best guess of what the age of the student might be,
as reflected from the completion. Choose from possible age groups: A: 3 or under. B: 4-5. C: 6-7. D: 8-9. E:
10-12. F: 13-16."""
    return eval_prompt1, eval_prompt2

In [ ]:
def get_evals(eval_model, lang, model_id):
    with open(f"answers_{lang}_{model_id}.json", "r") as f:
        answers = json.load(f)

    evaluations = []
    for a in tqdm(answers):
        eval_prompt1, eval_prompt2 = get_eval_prompts(a)
        eval_step1, eval_step2 = prompt_api(eval_model, eval_prompt1, eval_prompt2)
        a["eval_step1"] = eval_step1["choices"][0]["message"]["content"]
        a["eval_step2"] = eval_step2["choices"][0]["message"]["content"]
        evaluations.append(a)
    with open(f"evaluations_{lang}_{model_id}.json", "w") as f:
        json.dump(evaluations, f, indent="    ", ensure_ascii=False)

In [ ]:
eval_model = "gpt-4-1106-preview"
lang = "en"  # "en" / "de"
model_id = "gpt"  # "gpt" / "ts" / "own"

get_evals(eval_model, lang, model_id)

In [ ]:
# def get_single_score(eval_answers, id):
#     a = eval_answers[id]
#     eval_prompt = get_eval_prompt(a)
#     chat_completion = prompt_api(eval_model, eval_prompt)
#     return {
#         "id": a["id"],
#         "type": a["type"],
#         "question": a["question"],
#         "context": a["context"],
#         "answer": a["answer"],
#         "evaluation": chat_completion["choices"][0]["message"]["content"]
#     }

In [ ]:
# for id in tqdm(range(60)):
#     with open(f"evaluations_{model_id}.json", "r") as f:
#         evaluations = json.load(f)
#     evaluations[id] = get_single_score(answers, id)
#     with open(f"evaluations_{model_id}.json", "w") as f:
#         json.dump(evaluations, f, indent="    ")

In [7]:
# # SCRAPS FOR LATER
# template = "Once there was a dude name Doug. Doug loved to eat"
# completion = " strawberry icecream while riding his horse through Town."
# prompt1 = f"""In the following exercise, the student is given a beginning of a story. The student needs to complete it into a full story.
# The exercise tests the student´s language abilities and creativity. The symbol *** marks the separator between the
# prescribed beginning and the student’s completion:
# {template}***{completion}
# Please provide your general assessment about the part written by the student (the one after the *** symbol).
# Is it gramatically correct? Is it consistent with the beginning of the story? Pay special attention to whether the
# student manages to complete the sentence which is split in the middle by the separator ***.
# """

# prompt2 = f"""Now, grade the student’s completion in terms of grammar, creativity, consistency with the story’s beginning and
# whether the plot makes sense. Moreover, please provide your best guess of what the age of the student might be,
# as reflected from the completion. Choose from possible age groups: A: 3 or under. B: 4-5. C: 6-7. D: 8-9. E:
# 10-12. F: 13-16."""

# p1 = openai.ChatCompletion.create(
#     model = "gpt-3.5-turbo",
#     stream=False,
#     messages = [{"role": "user", "content": prompt1}]
# )

# for s in p1.choices[0].message.content.split(". "):
#     print(s + ".")

# print("--------------------------------------------------")

# p2 = openai.ChatCompletion.create(
#     model = "gpt-3.5-turbo",
#     stream=False,
#     messages = [
#         {"role": "user", "content": prompt1},
#         {"role": "assistant", "content": p1.choices[0].message.content},
#         {"role": "user", "content": prompt2},
#     ]
# )

# for s in p2.choices[0].message.content.split(". "):
#     print(s + ".")

APIError:  (request id: 20240320201348946562401bcDo2EbT) {"error":{"message":" (request id: 20240320201348946562401bcDo2EbT)","type":"upstream_error","param":"500","code":"bad_response_status_code"}} 500 {'error': {'message': ' (request id: 20240320201348946562401bcDo2EbT)', 'type': 'upstream_error', 'param': '500', 'code': 'bad_response_status_code'}} {'content-type': 'application/json; charset=utf-8', 'x-oneapi-request-id': '20240320201348946562401bcDo2EbT', 'content-length': '142', 'date': 'Wed, 20 Mar 2024 12:13:50 GMT', 'server': 'LiteSpeed', 'alt-svc': 'h3=":443"; ma=2592000, h3-29=":443"; ma=2592000, h3-Q050=":443"; ma=2592000, h3-Q046=":443"; ma=2592000, h3-Q043=":443"; ma=2592000, quic=":443"; ma=2592000; v="43,46"', 'connection': 'Keep-Alive'}

In [8]:
# story1 = "Write a short story about a flying Mango and his best friend a small log."
# story2 = "Continue above story incorporating a very fast Caterpillar."
# p1 = openai.ChatCompletion.create(
#     model = "gpt-3.5-turbo",
#     stream=False,
#     messages = [{"role": "user", "content": story1}]
# )

# for s in p1.choices[0].message.content.split(". "):
#     print(s + ".")

# print("--------------------------------------------------")

# p2 = openai.ChatCompletion.create(
#     model = "gpt-3.5-turbo",
#     stream=False,
#     messages = [
#         {"role": "user", "content": story1},
#         {"role": "assistant", "content": p1.choices[0].message.content},
#         {"role": "user", "content": story2},
#     ]
# )

# for s in p2.choices[0].message.content.split(". "):
#     print(s + ".")

Once upon a time, in a lush and vibrant forest, there lived a mango named Mo the Mango.
Mo was not your ordinary fruit, for he had a special ability – he could fly! With his bright yellow skin and wings made of leaves, Mo fluttered through the air, delighting all the creatures of the forest with his grace and charm.

Mo's best friend was a small log named Larry.
Larry may not have had wings like Mo, but he was sturdy and reliable, always there to support Mo in his adventures.
The two of them would spend their days exploring the forest, laughing and chatting as they soared through the treetops.

One day, as Mo and Larry were flying over a sparkling river, they spotted a family of squirrels in trouble.
A strong current had swept away their precious acorn stash, and they were frantically trying to gather them back up.
Without hesitation, Mo swooped down and scooped up the acorns in his leafy wings, carrying them back to the grateful squirrels.

From that day on, Mo and Larry became heroes

APIError:  (request id: 20240320201702332142325OYFG5VVr) {"error":{"message":" (request id: 20240320201702332142325OYFG5VVr)","type":"upstream_error","param":"500","code":"bad_response_status_code"}} 500 {'error': {'message': ' (request id: 20240320201702332142325OYFG5VVr)', 'type': 'upstream_error', 'param': '500', 'code': 'bad_response_status_code'}} {'content-type': 'application/json; charset=utf-8', 'x-oneapi-request-id': '20240320201702332142325OYFG5VVr', 'content-length': '142', 'date': 'Wed, 20 Mar 2024 12:17:04 GMT', 'server': 'LiteSpeed', 'alt-svc': 'h3=":443"; ma=2592000, h3-29=":443"; ma=2592000, h3-Q050=":443"; ma=2592000, h3-Q046=":443"; ma=2592000, h3-Q043=":443"; ma=2592000, quic=":443"; ma=2592000; v="43,46"', 'connection': 'Keep-Alive'}

In [ ]:
# story1 = "Write a short story about a flying Mango and his best friend a small log."
# story2 = "Continue above story incorporating a very fast Caterpillar."
# p1 = openai.ChatCompletion.create(
#     model = "gpt-3.5-turbo",
#     stream=False,
#     messages = [{"role": "user", "content": story1}]
# )

# for s in p1.choices[0].message.content.split(". "):
#     print(s + ".")

# print("--------------------------------------------------")

# p2 = openai.ChatCompletion.create(
#     model = "gpt-3.5-turbo",
#     stream=False,
#     messages = [
#         {"role": "user", "content": story1},
#         {"role": "assistant", "content": p1.choices[0].message.content},
#         {"role": "user", "content": story2},
#     ]
# )

# for s in p2.choices[0].message.content.split(". "):
#     print(s + ".")

In [6]:
# eval_step1 = {"choices": [{"message": {"content": "ERROR"}}]}
# print(eval_step1["choices"][0]["message"]["content"] == "ERROR")

True
